In [1]:
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
import xarray as xr


In [2]:
ndvi_ref_path = "../data/ndvi_daily/ndvi_day_01.tif"

with rasterio.open(ndvi_ref_path) as ref:
    ndvi_crs = ref.crs
    ndvi_transform = ref.transform
    ndvi_shape = ref.shape

print("NDVI shape:", ndvi_shape)
print("NDVI CRS:", ndvi_crs)


NDVI shape: (10342, 12242)
NDVI CRS: EPSG:4326


In [3]:
ndvi_stack = np.load("../data/ndvi_stack.npy")

print("NDVI stack shape:", ndvi_stack.shape)


NDVI stack shape: (14, 10342, 12242)


In [4]:
era5_path = "../data/era5_raw/era5_weather_si_20230101_20230114.nc"

ds = xr.open_dataset(era5_path)
ds = ds.rename({"valid_time": "time"})
ds_daily = ds.resample(time="1D").mean()


In [5]:
# Wind speed
ds_daily["wind_speed"] = np.sqrt(
    ds_daily["u10"]**2 + ds_daily["v10"]**2
)

# Relative Humidity
T = ds_daily["t2m"] - 273.15
Td = ds_daily["d2m"] - 273.15

es_T = 6.112 * np.exp((17.67 * T) / (T + 243.5))
es_Td = 6.112 * np.exp((17.67 * Td) / (Td + 243.5))

ds_daily["relative_humidity"] = 100 * (es_Td / es_T)


In [6]:
T_steps = ds_daily.dims["time"]
H, W = ndvi_shape

wind_aligned = np.zeros((T_steps, H, W), dtype=np.float32)
rh_aligned = np.zeros((T_steps, H, W), dtype=np.float32)


C:\Users\Haneesh.B\AppData\Local\Temp\ipykernel_36648\2169307018.py:1: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  T_steps = ds_daily.dims["time"]


In [7]:
for t in range(T_steps):
    wind_src = ds_daily["wind_speed"].isel(time=t).values
    rh_src = ds_daily["relative_humidity"].isel(time=t).values

    src_transform = rasterio.transform.from_bounds(
        float(ds.longitude.min()),
        float(ds.latitude.min()),
        float(ds.longitude.max()),
        float(ds.latitude.max()),
        wind_src.shape[1],
        wind_src.shape[0],
    )

    reproject(
        source=wind_src,
        destination=wind_aligned[t],
        src_transform=src_transform,
        src_crs="EPSG:4326",
        dst_transform=ndvi_transform,
        dst_crs=ndvi_crs,
        resampling=Resampling.bilinear
    )

    reproject(
        source=rh_src,
        destination=rh_aligned[t],
        src_transform=src_transform,
        src_crs="EPSG:4326",
        dst_transform=ndvi_transform,
        dst_crs=ndvi_crs,
        resampling=Resampling.bilinear
    )

    print(f"Aligned day {t+1}")


Aligned day 1
Aligned day 2
Aligned day 3
Aligned day 4
Aligned day 5
Aligned day 6
Aligned day 7
Aligned day 8
Aligned day 9
Aligned day 10
Aligned day 11
Aligned day 12
Aligned day 13
Aligned day 14


In [8]:
wind_aligned = wind_aligned[..., np.newaxis]
rh_aligned = rh_aligned[..., np.newaxis]

print(wind_aligned.shape, rh_aligned.shape)


(14, 10342, 12242, 1) (14, 10342, 12242, 1)


In [11]:
# Ensure NDVI has channel dimension
if ndvi_stack.ndim == 3:
    ndvi_stack = ndvi_stack[..., np.newaxis]

# Ensure wind & RH have channel dimension
if wind_aligned.ndim == 3:
    wind_aligned = wind_aligned[..., np.newaxis]

if rh_aligned.ndim == 3:
    rh_aligned = rh_aligned[..., np.newaxis]

print("NDVI shape:", ndvi_stack.shape)
print("Wind shape:", wind_aligned.shape)
print("RH shape:", rh_aligned.shape)


NDVI shape: (14, 10342, 12242, 1)
Wind shape: (14, 10342, 12242, 1)
RH shape: (14, 10342, 12242, 1)


In [14]:
def downsample(arr, factor=4):
    return arr[:, ::factor, ::factor, :]

# Downsample each tensor
ndvi_ds = downsample(ndvi_stack, factor=4)
wind_ds = downsample(wind_aligned, factor=4)
rh_ds = downsample(rh_aligned, factor=4)

print(ndvi_ds.shape, wind_ds.shape, rh_ds.shape)


(14, 2586, 3061, 1) (14, 2586, 3061, 1) (14, 2586, 3061, 1)


In [15]:
final_input = np.concatenate(
    (ndvi_ds, wind_ds, rh_ds),
    axis=-1
)

print("Final ConvLSTM input shape:", final_input.shape)


Final ConvLSTM input shape: (14, 2586, 3061, 3)


In [16]:
np.save("../data/convlstm_input_ds.npy", final_input)
print("Downsampled ConvLSTM input saved successfully")


Downsampled ConvLSTM input saved successfully
